In [1]:
from __future__ import print_function

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Lambda, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import csv
import os
import pickle

%matplotlib inline

In [2]:
new_size = 260

folder = 'CUB_200_2011'
images_folder = os.path.join(folder, 'images')

images_file = os.path.join(folder, 'images.txt')
split_file = os.path.join(folder, 'train_test_split.txt')
labels_file = os.path.join(folder, 'image_class_labels.txt')
classes_file = os.path.join(folder, 'classes.txt')

In [3]:
images = dict()

with open(images_file, 'r') as f:
    data = f.read()
data = data.strip().split('\n')

for line in data:
    num, name = line.split(' ')
    num = int(num)
    images[num] = name

In [4]:
splits = dict()

with open(split_file, 'r') as f:
    data = f.read()
data = data.strip().split('\n')

for line in data:
    num, split = line.split(' ')
    num = int(num)
    split = int(split)
    splits[num] = split

In [5]:
train_images = dict()
test_images = dict()

for key in images:
    if splits[key] == 1:
        train_images[key] = images[key]
    else:
        test_images[key] = images[key]

In [6]:
labels = dict()
with open(labels_file, 'r') as f:
    data = f.read()
data = data.strip().split('\n')

for line in data:
    num, label = line.split(' ')
    num = int(num)
    label = int(label)
    labels[num] = label

In [7]:
x_train = np.zeros((len(train_images), new_size, new_size, 3),np.float32)
x_test = np.zeros((len(test_images), new_size, new_size, 3),np.float32)
y_train = np.zeros(len(train_images), np.float32)
y_test = np.zeros(len(test_images), np.float32)

In [8]:
for i, key in enumerate(tqdm(train_images)):
    x = Image.open(os.path.join(images_folder, train_images[key])).convert('RGB').resize((new_size, new_size), Image.ANTIALIAS)
    x = np.array(x).astype('float32')
    x_train[i,:,:,:] = x
    
    y_train[i] = labels[key] - 1

for i, key in enumerate(tqdm(test_images)):
    x = Image.open(os.path.join(images_folder, test_images[key])).convert('RGB').resize((new_size, new_size), Image.ANTIALIAS)
    x = np.array(x).astype('float32')
    x_test[i,:,:,:] = x
    
    y_test[i] = labels[key] - 1

100%|██████████| 4874/4874 [00:17<00:00, 281.74it/s]


In [9]:
x_train = x_train/255
x_test = x_test/255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [11]:
####--------  PARAMETERS --------####

learning_rate = 0.005
num_epochs = 50
batch = 8
input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]
num_samples = y_train.shape[0]

In [12]:
####-------- MODEL DEFINITION --------####

conv_base = EfficientNetB2(weights='imagenet', include_top=False,
                           pooling='avg', input_shape=input_shape)

# for layer in conv_base.layers:
#     if isinstance(layer, BatchNormalization):
#         layer.trainable = True
#     else:
#         layer.trainable = False

model = Sequential()
model.add(conv_base)
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(4, activation="softmax"))

opt = Adam(learning_rate = learning_rate)
model.compile(optimizer = opt, loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [13]:
####-------- TRAINING --------####

def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lrSchedule = LearningRateScheduler(scheduler)

earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=10,
                              verbose=0,
                              mode='min',
                              restore_best_weights=True)

In [14]:
history = model.fit(x_train, y_train, validation_data = (x_test, y_test),
                    epochs = num_epochs, callbacks =[earlyStopping, lrSchedule],
                    batch_size = batch)

Epoch 1/50
2437/2437 [==============================] - 246s 98ms/step - loss: 1.1993 - accuracy: 0.5012 - val_loss: 2.3154 - val_accuracy: 0.5012
Epoch 2/50
2437/2437 [==============================] - 238s 98ms/step - loss: 0.8931 - accuracy: 0.6186 - val_loss: 45.8803 - val_accuracy: 0.1526
Epoch 3/50
2437/2437 [==============================] - 238s 98ms/step - loss: 0.8599 - accuracy: 0.6405 - val_loss: 1.4719 - val_accuracy: 0.5012
Epoch 4/50
2437/2437 [==============================] - 238s 98ms/step - loss: 1.0184 - accuracy: 0.5688 - val_loss: 4.7958 - val_accuracy: 0.5012
Epoch 5/50
2437/2437 [==============================] - 238s 98ms/step - loss: 0.7796 - accuracy: 0.6806 - val_loss: 2.3975 - val_accuracy: 0.5113
Epoch 6/50
2437/2437 [==============================] - 237s 97ms/step - loss: 0.7542 - accuracy: 0.6842 - val_loss: 3.0202 - val_accuracy: 0.5523
Epoch 7/50
2437/2437 [==============================] - 235s 96ms/step - loss: 0.7632 - accuracy: 0.6843 - val_loss: 

In [ ]:
name = 'efficientNetB2_batch_8.h5'
model.save(name)

In [15]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [24]:
save_obj(history.history, 'history')

In [10]:
name = 'efficientNetB2_batch_8.h5'
model = tf.keras.models.load_model(name)

In [12]:
y_pred = model.predict(x_train)
result = np.sum(np.argmax(y_train, axis=1) == np.argmax(y_pred, axis=1))/len(y_pred)
print(result)

0.9888672275805459


In [26]:
data = load_obj('history')

In [27]:
data.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr'])